In [1]:
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
from sklearn import datasets
from sklearn.base import BaseEstimator
from scipy.spatial import distance
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KDTree

In [2]:
LEAF_SIZE = 20


class MyKNeighborsClassifier(BaseEstimator):
    
    def __init__(self, n_neighbors, algorithm='brute'):
        self.n_neighbors = n_neighbors
        self.algorithm=algorithm

    def fit(self, X, y):
        if self.algorithm == 'brute':
            self.__brute__(X, y)
        if self.algorithm == 'kd_tree':
            self.__kdtree__(X, y)

    def __predict_brute__(self, X):
        dists = distance.cdist(X, self.X)
        def best(row):
            return np.argmax(np.bincount(self.y[np.argpartition(row, self.n_neighbors)[:self.n_neighbors]]))
        return np.apply_along_axis(best, 1, dists)

    def __predict_kdtree__(self, X):
        dists, inds = self.kdtree.query(X, k=self.n_neighbors)
        #print(dists)
        def best(ind):
            return np.argmax(np.bincount(self.y[ind]))
        return np.apply_along_axis(best, 1, inds)

    def predict(self, X):
        if self.algorithm == 'brute':
            return self.__predict_brute__(X)
        if self.algorithm == 'kd_tree':
            return self.__predict_kdtree__(X)
    
    def __brute__(self, X, y):
        self.X = X
        self.y = y

    def __kdtree__(self, X, y):
        self.y = y
        self.kdtree = KDTree(X, leaf_size=LEAF_SIZE, metric='euclidean')

In [3]:
iris = datasets.load_iris()

In [4]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [5]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')

In [6]:
%%time
clf.fit(X_train, y_train)

CPU times: user 1.35 ms, sys: 863 µs, total: 2.21 ms
Wall time: 2.49 ms


KNeighborsClassifier(algorithm='kd_tree', n_neighbors=2)

In [7]:
%%time
my_clf.fit(X_train, y_train)

CPU times: user 215 µs, sys: 48 µs, total: 263 µs
Wall time: 229 µs


In [10]:
sklearn_pred = clf.predict(X_test)
my_clf_pred = my_clf.predict(X_test)
sklearn_pred, my_clf_pred
assert abs( accuracy_score(y_test, my_clf_pred) -  accuracy_score(y_test, sklearn_pred ) )<0.005, "Score must be simillar"

In [12]:
%%time
my_clf.predict(X_test)

CPU times: user 2.22 ms, sys: 270 µs, total: 2.49 ms
Wall time: 1.48 ms


array([2, 2, 2, 0, 2, 0, 1, 0, 0, 1, 2, 2, 1, 0, 1])

In [13]:
%%time
clf.predict(X_test)

CPU times: user 2.76 ms, sys: 1.43 ms, total: 4.19 ms
Wall time: 2.88 ms


array([2, 2, 2, 0, 2, 0, 1, 0, 0, 1, 2, 2, 1, 0, 1])

In [14]:
%%time
assert(accuracy_score(my_clf.predict(X_test), y_test) > accuracy_score(clf.predict(X_test), y_test) - 0.005)

CPU times: user 3.59 ms, sys: 3.97 ms, total: 7.56 ms
Wall time: 5 ms
